In [48]:
from mako.template import Template

import argparse
import configparser
from itertools import chain
import re

from sys import platform as _platform
import os

In [77]:
DEBUG = True
IMAGE_NAME = 'quagga'
LINUX_TERMINAL_TYPE = 'xterm'

# Parsing of lab.conf

In [3]:
config = configparser.ConfigParser()

In [4]:
# reads lab.conf
with open('shared/MARACAS_lab/lab.conf') as stream:
    # adds a section to mimic a .ini file
    stream = chain(("[dummysection]",), stream)
    config.read_file(stream)

In [5]:
# gets 2 list of keys, one for machines and the other for the metadata
# we also need a unique list of links
keys = []
m_keys = []
links = []
for key in config['dummysection']: 
    if DEBUG: print(key, config['dummysection'][key])
    if '[' in key and ']' in key:
        keys.append(key)
        links.append(config['dummysection'][key])
    else:
        m_keys.append(key)

('lab_description', u'"absaaaaaaaaaaaaaaaaa"')
('lab_version', u'"1.0"')
('lab_author', u'"aaaaaaaaaaaaaaaa"')
('lab_email', u'"aaaaaaaaa@aaaaaaaaaaaaaa"')
('host[0]', u'A')
('server[0]', u'B')
('ospf1[0]', u'C')
('ospf1[1]', u'E')
('ospf1[2]', u'F')
('ospf2[0]', u'D')
('ospf2[1]', u'E')
('ospf3[0]', u'A')
('ospf3[1]', u'D')
('ospf3[2]', u'C')
('br[0]', u'G')
('br[1]', u'F')
('rip1[0]', u'H')
('rip1[1]', u'G')
('rip2[0]', u'B')
('rip2[1]', u'H')


In [6]:
config['dummysection'][keys[0]]

u'A'

In [7]:
# we only need unique links
links = set(links)

In [8]:
links

{u'A', u'B', u'C', u'D', u'E', u'F', u'G', u'H'}

In [9]:
# helper functions for natural sorting
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

# sort the keys so that we keep the order of the interfaces
keys.sort(key=natural_keys)

In [10]:
keys

['br[0]',
 'br[1]',
 'host[0]',
 'ospf1[0]',
 'ospf1[1]',
 'ospf1[2]',
 'ospf2[0]',
 'ospf2[1]',
 'ospf3[0]',
 'ospf3[1]',
 'ospf3[2]',
 'rip1[0]',
 'rip1[1]',
 'rip2[0]',
 'rip2[1]',
 'server[0]']

In [11]:
m_keys

['lab_description', 'lab_version', 'lab_author', 'lab_email']

In [12]:
# we then get a dictionary of machines ignoring interfaces that have missing positions (eg: 1,3,6 instead of 0,1,2)
machines = {}
for key in keys: 
    splitted = key.split('[')
    name = splitted[0]
    splitted = splitted[1].split(']')
    ifnumber = int(splitted[0])
    if not machines.get(name):
        machines[name] = []
    if len(machines[name]) == 0 or machines[name][len(machines[name])-1][1] == ifnumber - 1:
        machines[name].append((config['dummysection'][key], ifnumber))

In [13]:
machines

{'br': [(u'G', 0), (u'F', 1)],
 'host': [(u'A', 0)],
 'ospf1': [(u'C', 0), (u'E', 1), (u'F', 2)],
 'ospf2': [(u'D', 0), (u'E', 1)],
 'ospf3': [(u'A', 0), (u'D', 1), (u'C', 2)],
 'rip1': [(u'H', 0), (u'G', 1)],
 'rip2': [(u'B', 0), (u'H', 1)],
 'server': [(u'B', 0)]}

In [14]:
metadata = {}
for m_key in m_keys:
    if config['dummysection'][m_key].startswith('"') and config['dummysection'][m_key].endswith('"'):
        config['dummysection'][m_key] = config['dummysection'][m_key][1:-1]
    metadata[m_key] = config['dummysection'][m_key]

In [15]:
metadata

{'lab_author': u'aaaaaaaaaaaaaaaa',
 'lab_description': u'absaaaaaaaaaaaaaaaaa',
 'lab_email': u'aaaaaaaaa@aaaaaaaaaaaaaa',
 'lab_version': u'1.0'}

# Generation of docker-compose.yml (may not be used in the end)

In [16]:
network_block_template = '''networks: 
${links}
'''
network_link_template = '''    ${link}:
'''
service_block_template = '''version: '2.1'

services:
${services}
'''
service_template = '''    ${machine_name}:
        image: ${image_name}
        privileged: true
        container_name: netkit-${machine_name}
        ports:
            - 808${number}:80
        networks:
${links}
'''
service_link_template = '''            ${link}:
'''
# TODO lab.dep with '''depends_on: service: condition: service_healty'''

In [17]:
print(Template('''hello ${data}! 
               ''').render(data="world"))

hello world! 
               


In [18]:
conf_network = ""
for link in links:
    conf_network = conf_network + Template(network_link_template).render(link=link)

conf_network = Template(network_block_template).render(links=conf_network)

In [19]:
print(conf_network)

networks: 
    A:
    C:
    B:
    E:
    D:
    G:
    F:
    H:




In [20]:
conf_services = ""
counter = 0
for machine_name, interfaces in machines.items():
    conf_links = ""
    for link, _ in interfaces:
        conf_links = conf_links + Template(service_link_template).render(link=link)
    conf_services = conf_services + Template(service_template).render(machine_name=machine_name, image_name=IMAGE_NAME, number=counter, links=conf_links)
    counter += 1
    
conf_services = Template(service_block_template).render(services=conf_services)

In [21]:
print(conf_services)

version: '2.1'

services:
    host:
        image: quagga
        privileged: true
        container_name: netkit-host
        ports:
            - 8080:80
        networks:
            A:

    server:
        image: quagga
        privileged: true
        container_name: netkit-server
        ports:
            - 8081:80
        networks:
            B:

    rip2:
        image: quagga
        privileged: true
        container_name: netkit-rip2
        ports:
            - 8082:80
        networks:
            B:
            H:

    rip1:
        image: quagga
        privileged: true
        container_name: netkit-rip1
        ports:
            - 8083:80
        networks:
            H:
            G:

    br:
        image: quagga
        privileged: true
        container_name: netkit-br
        ports:
            - 8084:80
        networks:
            G:
            F:

    ospf1:
        image: quagga
        privileged: true
        container_name: netkit-ospf1
        ports:

# Generation of docker run and docker network commands (lstart)

In [66]:
create_network_template = 'docker network create '
create_network_commands = []
for link in links:
    create_network_commands.append(create_network_template + link)

In [68]:
create_machine_template = 'docker run --name {machine_name} -p 808{number}:80 --network={first_link} {image_name}'
# we could use -ti -a stdin -a stdout and then /bin/bash -c "commands;bash", 
# but that woult execute commands like ifconfig BEFORE all the networks are linked
create_machine_commands = []

create_connection_template = 'docker network connect {link} {machine_name}'
create_connection_commands = []

copy_folder_template = 'docker cp {machine_name}/etc {machine_name}:/etc'
copy_folder_commands = []

count = 0
def replace_multiple_items(repls, string):
    return reduce(lambda a, kv: a.replace(*kv), repls, string)

for machine_name, interfaces in machines.items():
    repls = ('{machine_name}', machine_name), ('{number}', str(count)), ('{first_link}', interfaces[0][0]), ('{image_name}', IMAGE_NAME)
    create_machine_commands.append(replace_multiple_items(repls, create_machine_template))
    count += 1
    for link,_ in interfaces[1:]:
        repls = ('{link}', link), ('{machine_name}', machine_name)
        create_connection_commands.append(replace_multiple_items(repls, create_connection_template))
    repls = ('{machine_name}', machine_name), ('{machine_name}', machine_name)
    copy_folder_commands.append(replace_multiple_items(repls, copy_folder_template))

In [70]:
print create_network_commands
print create_machine_commands
print create_connection_commands
print copy_folder_commands

[u'docker network create A', u'docker network create C', u'docker network create B', u'docker network create E', u'docker network create D', u'docker network create G', u'docker network create F', u'docker network create H']
[u'docker run --name host -p 8080:80 --network=A quagga', u'docker run --name server -p 8081:80 --network=B quagga', u'docker run --name rip2 -p 8082:80 --network=B quagga', u'docker run --name rip1 -p 8083:80 --network=H quagga', u'docker run --name br -p 8084:80 --network=G quagga', u'docker run --name ospf1 -p 8085:80 --network=C quagga', u'docker run --name ospf3 -p 8086:80 --network=A quagga', u'docker run --name ospf2 -p 8087:80 --network=D quagga']
[u'docker network connect H rip2', u'docker network connect G rip1', u'docker network connect F br', u'docker network connect E ospf1', u'docker network connect F ospf1', u'docker network connect D ospf3', u'docker network connect C ospf3', u'docker network connect E ospf2']
['docker cp host/etc host:/etc', 'docke

In [71]:
commands = create_network_commands + create_machine_commands + create_connection_commands + copy_folder_commands

In [72]:
print commands

[u'docker network create A', u'docker network create C', u'docker network create B', u'docker network create E', u'docker network create D', u'docker network create G', u'docker network create F', u'docker network create H', u'docker run --name host -p 8080:80 --network=A quagga', u'docker run --name server -p 8081:80 --network=B quagga', u'docker run --name rip2 -p 8082:80 --network=B quagga', u'docker run --name rip1 -p 8083:80 --network=H quagga', u'docker run --name br -p 8084:80 --network=G quagga', u'docker run --name ospf1 -p 8085:80 --network=C quagga', u'docker run --name ospf3 -p 8086:80 --network=A quagga', u'docker run --name ospf2 -p 8087:80 --network=D quagga', u'docker network connect H rip2', u'docker network connect G rip1', u'docker network connect F br', u'docker network connect E ospf1', u'docker network connect F ospf1', u'docker network connect D ospf3', u'docker network connect C ospf3', u'docker network connect E ospf2', 'docker cp host/etc host:/etc', 'docker c

# Run command ang get output

In [45]:
import subprocess
def run_command_detatched(cmd_line):
    p = subprocess.Popen(cmd_line, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    out = p.communicate()[0]
    return out

In [46]:
print run_command_detatched('ping www.google.com')


Esecuzione di Ping www.google.com [2a00:1450:4002:800::2004] con 32 byte di dati:
Risposta da 2a00:1450:4002:800::2004: durata=88ms 
Risposta da 2a00:1450:4002:800::2004: durata=81ms 
Risposta da 2a00:1450:4002:800::2004: durata=80ms 
Richiesta scaduta.

Statistiche Ping per 2a00:1450:4002:800::2004:
    Pacchetti: Trasmessi = 4, Ricevuti = 3, 
    Persi = 1 (25% persi),
Tempo approssimativo percorsi andata/ritorno in millisecondi:
    Minimo = 80ms, Massimo =  88ms, Medio =  83ms



# Full command generation

In [42]:
if _platform == "linux" or _platform == "linux2":
    print 'linux'
elif _platform == "darwin":
    print 'MAC OS X'
elif _platform == "win32":
    print 'Windows'

Windows


In [58]:
os.system('start /wait cmd /c "ping www.google.com & cmd.exe"')

0

In [59]:
os.system(LINUX_TERMINAL_TYPE + ' -e \'bash -c "ping www.google.com; exec bash"\'')

1

In [60]:
os.system('Terminal.app -c "ping www.google.com; exec Terminal.app"') # not tested yet

1

In [78]:
open_bash_template_linux = LINUX_TERMINAL_TYPE + ' -e \'bash -c "{commands} exec bash"\''
open_bash_template_windows = 'start /wait cmd /c "{commands} cmd.exe"'
open_bash_template_osx = 'Terminal.app -c "{commands} exec Terminal.app"'  # not tested yet

separator_unix = ' ; '
separator_windows = ' & '

open_bash_template = ''
open_bash_separator = ''

if _platform == "win32":
    open_bash_template = open_bash_template_windows
    open_bash_separator = separator_windows
elif _platform == "darwin":
    open_bash_template = open_bash_template_osx
    open_bash_separator = separator_unix
else: #linux or linux2
    open_bash_template = open_bash_template_linux
    open_bash_separator = separator_unix

In [79]:
command_string = ''
for command in commands:
    command_string += command + open_bash_separator
if DEBUG: print open_bash_template.replace('{commands}', command_string)

start /wait cmd /c "docker network create A & docker network create C & docker network create B & docker network create E & docker network create D & docker network create G & docker network create F & docker network create H & docker run --name host -p 8080:80 --network=A quagga & docker run --name server -p 8081:80 --network=B quagga & docker run --name rip2 -p 8082:80 --network=B quagga & docker run --name rip1 -p 8083:80 --network=H quagga & docker run --name br -p 8084:80 --network=G quagga & docker run --name ospf1 -p 8085:80 --network=C quagga & docker run --name ospf3 -p 8086:80 --network=A quagga & docker run --name ospf2 -p 8087:80 --network=D quagga & docker network connect H rip2 & docker network connect G rip1 & docker network connect F br & docker network connect E ospf1 & docker network connect F ospf1 & docker network connect D ospf3 & docker network connect C ospf3 & docker network connect E ospf2 & docker cp host/etc host:/etc & docker cp server/etc server:/etc & dock

In [80]:
os.system(open_bash_template.replace('{commands}', command_string))

0